In [94]:
import swoq_pb2
import replay_pb2
import grpc
from pathlib import Path

In [95]:
replay = Path(r'D:\Projects\swoq-stuff\Replays\Quest\Ralph - a24f34e8-0b24-4420-ae30-8d8ecf33a3b8.bin')

In [96]:
from google.protobuf.internal.decoder import _DecodeVarint32

In [97]:
def analyze_player_state(playerState: swoq_pb2.PlayerState, level: int) -> None:
    global tiles_seen_at, inventory_seen_at
    
    for tile in playerState.surroundings:
        if tile not in tiles_seen_at:
            tiles_seen_at[tile] = level
            
    if playerState.inventory not in inventory_seen_at:
        inventory_seen_at[playerState.inventory] = level


def analyze_state(state: swoq_pb2.State) -> None:
    global second_player_at
    
    level = state.level
    
    if state.HasField('playerState'):
        analyze_player_state(state.playerState, level)
    if state.HasField('player2State'):
        analyze_player_state(state.player2State, level)
        if second_player_at is None:
            second_player_at = level

In [ ]:
tiles_seen_at = {}
inventory_seen_at = {}
second_player_at = None

pos, next_pos = 0, 0
with open(replay, 'rb') as file:
    data = file.read()

header = None
start_req = None
start_rsp = None
act_req = None
act_rsp = None

while pos < len(data):
    next_pos, pos = _DecodeVarint32(data, pos)
    msg = data[pos:pos+next_pos]
    
    if header is None:
        header = replay_pb2.ReplayHeader()
        header.ParseFromString(msg)
    elif start_req is None:
        start_req = swoq_pb2.StartRequest()
        start_req.ParseFromString(msg)
    elif start_rsp is None:
        start_rsp = swoq_pb2.StartResponse()
        start_rsp.ParseFromString(msg)
        analyze_state(start_rsp.state)
    elif act_req is None:
        act_req = swoq_pb2.ActRequest()
        act_req.ParseFromString(msg)
    elif act_rsp is None:
        act_rsp = swoq_pb2.ActResponse()
        act_rsp.ParseFromString(msg)
        analyze_state(act_rsp.state)
        act_req = None
        act_rsp = None
    pos += next_pos

f'{second_player_at=}'

In [ ]:
for t in swoq_pb2.Tile.values():
    print(swoq_pb2.Tile.Name(t), tiles_seen_at[t] if t in tiles_seen_at else '?')
print()
for i in swoq_pb2.Inventory.values():
    print(swoq_pb2.Inventory.Name(i), inventory_seen_at[i] if i in inventory_seen_at else '?')